## El crecimiento de la tasa de criminalidad en las principales ciudades de España no es por causa de los inmigrantes. 

### preguntas de investigación
1. ¿han aumentado los crímenes?
2. ¿Cuál es la nacionalidad de los condenados por crímenes?
3. ¿en que se ocupan los migrantes?
4. ¿beneficio de la migracion? 
5. ¿Qué trabajos hacen los inmigrantes?
6. ¿qué porcentaje de la población, latina y extranjera, han realizado crímenes?

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Cargar datos
activos = pd.read_csv("../datos/activos_nacionalidad_sexo_edad.csv")
empleo = pd.read_csv("../datos/tasa_empleo_nacionalidad_sexo.csv")
delitos = pd.read_csv("../datos/delitos_nacionalidad.csv")
condenados = pd.read_csv("../datos/condenados_sexo_nacionalidad_numero_delitos.csv")
ocupados = pd.read_csv("../datos/ocupados_nacionalidad_sector.csv")
poblacion = pd.read_csv("../datos/poblacion_pais_nacimiento.csv")
salario = pd.read_csv("../datos/salarios_jornada_nacionalidad_decil.csv")


In [3]:
activos

,Sexo,Edad,Nacionalidad,Periodo,Total
0,Ambos sexos,Total,Total,2024,"24.424,6"
1,Ambos sexos,Total,Total,2023,"24.119,7"
2,Ambos sexos,Total,Total,2022,"23.626,7"
3,Ambos sexos,Total,Total,2021,"23.309,9"
4,Ambos sexos,Total,Total,2020,"22.733,3"
...,...,...,...,...,...
2731,Mujeres,55 y más años,Extranjera: Resto del mundo y apátrida,2010,"9,4"
2732,Mujeres,55 y más años,Extranjera: Resto del mundo y apátrida,2009,"7,9"
2733,Mujeres,55 y más años,Extranjera: Resto del mundo y apátrida,2008,"4,8"
2734,Mujeres,55 y más años,Extranjera: Resto del mundo y apátrida,2007,"2,3"
